**Import des données**

In [66]:
# 📦 Imports
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_parquet('../data/parquet/full_2020.csv.parquet', engine='pyarrow')

**Optimisation**

In [67]:
# Liste des colonnes à supprimer (remplissage < 10%)
cols_to_drop_1 = [
    'ancien_code_commune', 'ancien_nom_commune', 'ancien_id_parcelle',
    'lot1_numero', 'lot1_surface_carrez', 'lot2_numero', 'lot2_surface_carrez',
    'lot3_numero', 'lot3_surface_carrez', 'lot4_numero', 'lot4_surface_carrez',
    'lot5_numero', 'lot5_surface_carrez'
]
# Liste des colonnes avec infos inutiles
cols_to_drop_2 = [
    'id_mutation', 'numero_disposition', 'adresse_numero', 'adresse_suffixe',
    'adresse_nom_voie', 'adresse_code_voie', 'nom_commune', 'id_parcelle',
    'numero_volume', 'code_nature_culture', 'nature_culture','code_nature_culture_speciale'
]
df.drop(columns=cols_to_drop_1, inplace=True)
# Vérification des colonnes restantes
print(f"Shape after dropping columns: {df.shape}")

def optimize_dataframe(df, parse_dates=None, category_thresh=0.05, verbose=True):
    """
    Optimise les types d'un DataFrame pour réduire l'utilisation mémoire :
    - convertit les objets en catégories si nombre de modalités faible
    - convertit les float64 en float32
    - convertit les int64 en int32
    - convertit les colonnes de dates

    Parameters:
    - df : DataFrame à optimiser
    - parse_dates : liste de colonnes à parser comme dates
    - category_thresh : seuil max de ratio modalité/nb lignes pour transformer en 'category'
    - verbose : affiche la mémoire gagnée

    Returns:
    - df optimisé
    """

    initial_memory = df.memory_usage(deep=True).sum() / 1024**2

    # Dates
    if parse_dates:
        for col in parse_dates:
            df[col] = pd.to_datetime(df[col], errors='coerce')

    # Float → float32
    float_cols = df.select_dtypes(include=['float64']).columns
    for col in float_cols:
        df[col] = df[col].astype('float32')

    # Int → int32
    int_cols = df.select_dtypes(include=['int64']).columns
    for col in int_cols:
        if df[col].isnull().any():
            df[col] = df[col].astype('Int32')
        else:
            df[col] = df[col].astype('int32')

    # Object → category si peu de modalités
    for col in df.select_dtypes(include='object').columns:
        if df[col].nunique(dropna=False) / len(df) <= category_thresh:
            df[col] = df[col].astype('category')

    final_memory = df.memory_usage(deep=True).sum() / 1024**2

    if verbose:
        print(f"💾 Mémoire utilisée : {initial_memory:.2f} Mo → {final_memory:.2f} Mo ({100 * (1 - final_memory/initial_memory):.1f}% gagné)")

    return df

# Application test sur ech_annonces_ventes_68.csv
optimized_df1 = optimize_dataframe(df, parse_dates=['date_mutation'], verbose=True)
optimized_df1.info()



Shape after dropping columns: (3522416, 27)
💾 Mémoire utilisée : 2855.43 Mo → 862.50 Mo (69.8% gagné)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3522416 entries, 0 to 3522415
Data columns (total 27 columns):
 #   Column                        Dtype         
---  ------                        -----         
 0   id_mutation                   object        
 1   date_mutation                 datetime64[ns]
 2   numero_disposition            int32         
 3   nature_mutation               category      
 4   valeur_fonciere               float32       
 5   adresse_numero                float32       
 6   adresse_suffixe               category      
 7   adresse_nom_voie              object        
 8   adresse_code_voie             category      
 9   code_postal                   float32       
 10  code_commune                  category      
 11  nom_commune                   category      
 12  code_departement              category      
 13  id_parcelle                   

**Pré processing**

In [ ]:
# optimized_df1 = optimized_df1.dropna()
optimized_df1.head()

# Le but est de voir si la commune est grande ou petite en fonction du nombre d'apparition dans le tableau
commune_freq = optimized_df1['nom_commune'].value_counts()
optimized_df1['commune_freq'] = optimized_df1['nom_commune'].map(commune_freq)



# code_type_local	type_local  nature_mutation   code_nature_culture	nature_culture  code_nature_culture_speciale   à encoder en 12345 !

# numero_volume 847 val différentes est ce qu'on supprime ???

# code_nature_culture_speciale -> 121 valeurs 'ABREU', 'ABRIC', 'ACACI', 'AEROD', ..., 'VAOC', 'VDQS', 'VIGNE', 'VIVIE']
# nature_culture_speciale Même valeurs que le code mais sans être des abréviations 


optimized_df1 = optimized_df1.drop(['adresse_numero', 'adresse_suffixe', 'adresse_nom_voie','adresse_code_voie', 'code_postal', 'nom_commune'], axis=1)

optimized_df1.head()




,id_mutation,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,code_commune,code_departement,id_parcelle,numero_volume,nombre_lots,code_type_local,type_local,surface_reelle_bati,nombre_pieces_principales,code_nature_culture,nature_culture,code_nature_culture_speciale,nature_culture_speciale,surface_terrain,longitude,latitude,commune_encoded,commune_freq
0,2020-1,2020-01-07,1,Vente,8000.0,01072,01,01072000AK0216,NaN,0,NaN,NaN,NaN,NaN,T,terres,NaN,NaN,1061.0,5.323532,46.171940,279476.531250,169
1,2020-2,2020-01-02,1,Vente,2175.0,01203,01,012030000B0004,NaN,0,NaN,NaN,NaN,NaN,BT,taillis simples,NaN,NaN,85.0,4.893454,46.251858,207441.234375,88
2,2020-2,2020-01-02,1,Vente,2175.0,01203,01,012030000B0173,NaN,0,NaN,NaN,NaN,NaN,T,terres,NaN,NaN,1115.0,4.900210,46.235275,207441.234375,88
3,2020-2,2020-01-02,1,Vente,2175.0,01203,01,012030000B0477,NaN,0,NaN,NaN,NaN,NaN,T,terres,NaN,NaN,1940.0,4.882112,46.246555,207441.234375,88
4,2020-2,2020-01-02,1,Vente,2175.0,01203,01,012030000C0068,NaN,0,NaN,NaN,NaN,NaN,T,terres,NaN,NaN,1148.0,4.894481,46.251842,207441.234375,88
